In [123]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.integrate import quad
%matplotlib inline

low_indexes  = ['₀', '₁', '₂', '₃', '₄', '₅']


x = [[83, 85],
    [84,85,85,86,86,87],
    [86,87,87,87,88,88,88,88,88,89,90],
    [89,90,90,91],
    [90,92]]

    
x1 = [1,0,0,0,0]
x2 = [0,1,0,0,0]
x3 = [0,0,1,0,0]
x4 = [0,0,0,1,0]
x5 = [0,0,0,0,1]


# x = [[1,2,3,1,0,2,1],
#      [2,3,2,1,4],
#      [4,5,3]]

# x1 = [1,0,0]
# x2 = [0,1,0]
# x3 = [0,0,1]


volume = 0
for el in x:
    volume += len(el)
alpha = 0.05
p = len(x)

# PSI = np.array([x1 for i in range(len(x[0]))] + [x2 for i in range(len(x[1]))] + [x3 for i in range(len(x[2]))])
# Y = np.array([el for ell in x for el in ell]).T

class linear_resression():
    def __init__(self, Y, PSI):
        self.Y = Y
        self.PSI = PSI
        self._calc()
    
    def _calc(self):
        self.Finv = np.linalg.inv(self.PSI.T.dot(self.PSI))
        self.beta = self.Finv.dot(self.PSI.T).dot(self.Y)
        self.e = self.Y - self.PSI.dot(self.beta)
        self.RSS = self.e.T.dot(self.e)

        y_mean = sum(self.Y)/len(self.Y)
        TSS = 0
        for el in self.Y:
            TSS += (el - y_mean)**2
        self.TSS = TSS
        self.R2 = (self.TSS - self.RSS)/self.TSS



## Нахождение уравнения линейной регрессии

In [125]:
PSI = np.array([x1 for i in range(len(x[0]))] + [x2 for i in range(len(x[1]))] + [x3 for i in range(len(x[2]))] + \
        [x4 for i in range(len(x[3]))] + [x5 for i in range(len(x[4]))])
                                           
Y = np.array([el for ell in x for el in ell])

def format_equation(beta):
    ret = f"Уравнение регрессии: y = {round(beta[0], 2)}∙x{low_indexes[1]}"
    for i in range(1, len(beta)):
        number = round(beta[i], 2)
        if number > 0:
            ret += f" + {number}∙x{low_indexes[i+1]}"
        else:
            ret += f" - {-number}∙x{low_indexes[i+1]}"
    return ret

main_regr = linear_resression(PSI = PSI, Y = Y)
print(format_equation(main_regr.beta))


Уравнение регрессии: y = 84.0∙x₁ + 85.5∙x₂ + 87.82∙x₃ + 90.0∙x₄ + 91.0∙x₅


## Проверка значимости коэффициентов регрессии

In [126]:
def delta_assessment_b2(delta_index):
    if delta_index >= p:
        raise RuntimeError(f"Индекс коэффициента регрессии может принимать только значения от 0 до {p-1}!")
    return main_regr.beta[delta_index]/(main_regr.RSS*main_regr.Finv[delta_index][delta_index])**0.5*(volume - p)**0.5

def get_pvalue_b2(fr): # распределение Стьюдента
    fr = abs(fr)
    k = volume-p
    def f(x):
        return math.gamma((k+1)/2)/((k*math.pi)**0.5 * math.gamma(k/2) * (1 + x**2/k)**((k+1)/2))
    ret, _ = quad(f, fr, math.inf)
    return 2*ret


for i in range(p):
    delta = delta_assessment_b2(i)
    pval = get_pvalue_b2(delta)
    if pval > alpha:
        print(f"Коэффициент перед x{low_indexes[i+1]} не является значимым")
    else:
        print(f"Коэффициент x{low_indexes[i+1]} значим!")


Коэффициент x₁ значим!
Коэффициент x₂ значим!
Коэффициент x₃ значим!
Коэффициент x₄ значим!
Коэффициент x₅ значим!


## Некоторые параметры регрессии

In [127]:
print(f"R² = {round(main_regr.R2, 3)}")
print(f"RSS = {round(main_regr.RSS, 3)}")
print(f"TSS = {round(main_regr.TSS, 3)}")

R² = 0.811
RSS = 23.136
TSS = 122.16


## Попарное сравнение однородности групп


## $H_0: \beta_i = \beta_j$
## $H_1: \beta_i \ne \beta_j$
## $\Delta = \frac{\widetilde\beta_i - \widetilde\beta_j}{ \sqrt{ RSS( F_{ii}^{-1} + F_{jj}^{-1}} )} \sqrt{n-p} \sim t(n-p)$

In [128]:
def check_uniformity(a,b):
    coef = (volume - p)**0.5 / (main_regr.RSS * (main_regr.Finv[a][a] + main_regr.Finv[b][b]))**0.5
    delta_assessment = (main_regr.beta[a] - main_regr.beta[b]) * coef
    pval = get_pvalue_b2(delta_assessment)    

    if pval < alpha:
        print(f"Группы {a+1} и {b+1} нельзя считать однородными!")
    else:
        print(f"Группы {a+1} и {b+1} можно считать однородными")

for i in range(p):
    for j in range(i+1,p):
        check_uniformity(i, j)

Группы 1 и 2 можно считать однородными
Группы 1 и 3 нельзя считать однородными!
Группы 1 и 4 нельзя считать однородными!
Группы 1 и 5 нельзя считать однородными!
Группы 2 и 3 нельзя считать однородными!
Группы 2 и 4 нельзя считать однородными!
Группы 2 и 5 нельзя считать однородными!
Группы 3 и 4 нельзя считать однородными!
Группы 3 и 5 нельзя считать однородными!
Группы 4 и 5 можно считать однородными
